# **Лабораторная работа #2**
## **Задание #1**


In [13]:
import sys
import math


def analyze_file(file_path):
    try:
        # Открываем файл в бинарном режиме для побайтового чтения
        with open(file_path, 'rb') as f:
            data = f.read()
    except FileNotFoundError:
        print(f"Ошибка: Файл '{file_path}' не найден.")
        sys.exit(1)

    n = len(data)  # Длина файла в байтах
    count = [0] * 256  # Массив для хранения частот каждого байта
    for byte in data:
        count[byte] += 1

    total_info_bits = 0.0
    symbol_stats = []

    for i in range(256):
        if count[i] > 0:
            # Вероятность символа
            p = count[i] / n
            # Количество информации в символе (в битах)
            info_per_symbol = -math.log2(p)
            total_info_bits += info_per_symbol * count[i]
            symbol_stats.append({
                'byte': i,
                'count': count[i],
                'p': p,
                'info': info_per_symbol
            })

    print("--- Результаты анализа файла ---")
    print(f"Имя файла: {file_path}")
    print(f"Длина файла (n): {n} байт")

    print("\n--- Сравнение в битах ---")
    L_bits = n * 8
    print(f"Длина файла L(Q) = n * 8 = {L_bits} бит")
    print(f"Оценка суммарного количества информации I_BP(Q) = {total_info_bits:.2f} бит")
    # Выводим дробную часть в экспоненциальной форме
    fractional_part = total_info_bits - int(total_info_bits)
    print(f"Дробная часть I_BP(Q) = {fractional_part:.2e}")

    print("\n--- Сравнение в октетах и оценка сжатия ---")
    I_octets = total_info_bits / 8
    E = math.ceil(I_octets)
    G_64 = E + 256 * 8
    G_8 = E + 256 * 1

    print(f"Длина файла L(Q) = n = {n} октетов")
    print(
        f"Оценка информации I_BP(Q) = I_BP(Q)[бит] / 8 = {I_octets:.2f} октетов")
    print(
        f"Оценка снизу длины сжатого текста E = ceil(I_BP(Q)[октетов]) = {int(E)} октетов")
    print(
        f"Оценка длины архива G_64 (с 64-битными частотами) = E + 256 * 8 = {int(G_64)} октетов")
    print(
        f"Оценка длины архива G_8 (с 8-битными частотами) = E + 256 * 1 = {int(G_8)} октетов")

    print("\n--- Таблица характеристик символов (сортировка по байту) ---")
    print("Байт (hex) | Частота | Вероятность | Информация (бит)")
    print("-" * 55)
    for stat in sorted(symbol_stats, key=lambda x: x['byte']):
        print(
            f"  {stat['byte']:02x}      | {stat['count']:^7} | {stat['p']:.6f}  | {stat['info']:.4f}")

    print("\n--- Таблица характеристик символов (сортировка по убыванию частоты) ---")
    print("Байт (hex) | Частота | Вероятность | Информация (бит)")
    print("-" * 55)
    for stat in sorted(symbol_stats, key=lambda x: x['count'], reverse=True):
        print(
            f"  {stat['byte']:02x}      | {stat['count']:^7} | {stat['p']:.6f}  | {stat['info']:.4f}")


analyze_file("utf8.txt")

--- Результаты анализа файла ---
Имя файла: utf8.txt
Длина файла (n): 764445 байт

--- Сравнение в битах ---
Длина файла L(Q) = n * 8 = 6115560 бит
Оценка суммарного количества информации I_BP(Q) = 3407055.23 бит
Дробная часть I_BP(Q) = 2.34e-01

--- Сравнение в октетах и оценка сжатия ---
Длина файла L(Q) = n = 764445 октетов
Оценка информации I_BP(Q) = I_BP(Q)[бит] / 8 = 425881.90 октетов
Оценка снизу длины сжатого текста E = ceil(I_BP(Q)[октетов]) = 425882 октетов
Оценка длины архива G_64 (с 64-битными частотами) = E + 256 * 8 = 427930 октетов
Оценка длины архива G_8 (с 8-битными частотами) = E + 256 * 1 = 426138 октетов

--- Таблица характеристик символов (сортировка по байту) ---
Байт (hex) | Частота | Вероятность | Информация (бит)
-------------------------------------------------------
  0a      |  13168  | 0.017226  | 5.8593
  20      |  70419  | 0.092118  | 3.4404
  21      |   306   | 0.000400  | 11.2867
  22      |  1079   | 0.001411  | 9.4686
  23      |   119   | 0.000156 

## **Задание 2**

In [14]:
def analyze_unicode_file(file_path):
    try:
        # Открываем файл в текстовом режиме с кодировкой UTF-8
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            text = f.read()
    except FileNotFoundError:
        print(f"Ошибка: Файл '{file_path}' не найден.")
        sys.exit(1)

    n = len(text)
    symbol_counts = {}
    for char in text:
        symbol_counts[char] = symbol_counts.get(char, 0) + 1

    unique_symbols_count = len(symbol_counts)

    # Расчёт общего размера символов в байтах
    total_symbol_bytes = 0
    for char in symbol_counts:
        # Кодируем каждый символ в UTF-8 и считаем длину в байтах
        total_symbol_bytes += len(char.encode('utf-8'))

    # Средний размер символа в байтах
    avg_symbol_size = total_symbol_bytes / unique_symbols_count if unique_symbols_count > 0 else 0

    # Расчёты
    total_info_bits = 0.0
    symbol_stats = []

    for char, count in symbol_counts.items():
        p = count / n
        if p > 0:
            info_per_symbol = -math.log2(p)
            total_info_bits += info_per_symbol * count
            symbol_stats.append({
                'char': char,
                'count': count,
                'p': p,
                'info': info_per_symbol
            })

    print("--- Результаты анализа Unicode файла ---")
    print(f"Имя файла: {file_path}")
    print(f"Длина файла (n): {n} символов")
    print(f"Размер алфавита |A1|: {unique_symbols_count} символов")
    print(f"Точный средний размер символа: {avg_symbol_size:.2f} байт/символ")

    # Оценка в битах
    print("\n--- Оценка в битах ---")
    print(f"Оценка суммарного количества информации I_BP(Q) = {total_info_bits:.2f} бит")
    fractional_part = total_info_bits - int(total_info_bits)
    print(f"Дробная часть {{I_BP(Q)}} = {fractional_part:.2e}")

    # Оценка длины архива
    print("\n--- Оценка длины архива ---")
    I_octets = total_info_bits / 8
    E = math.ceil(I_octets)
    
    table_length = unique_symbols_count * avg_symbol_size + unique_symbols_count * 8
    G_arch = E + table_length

    print(f"Оценка информации I_BP(Q) = {I_octets:.2f} октетов")
    print(f"Оценка снизу длины сжатого текста E = {int(E)} октетов")
    print(f"Оценка длины архива G_arch = {G_arch:.2f} октетов")

    print("\n--- Таблица характеристик символов (сортировка по символу) ---")
    print("Символ | Частота | Вероятность | Информация (бит)")
    print("-" * 60)
    for stat in sorted(symbol_stats, key=lambda x: x['char']):
        print(f"  {stat['char']:<4} | {stat['count']:^7} | {stat['p']:.6f}  | {stat['info']:.4f}")

    print("\n--- Таблица характеристик символов (сортировка по убыванию частоты) ---")
    print("Символ | Частота | Вероятность | Информация (бит)")
    print("-" * 60)
    for stat in sorted(symbol_stats, key=lambda x: x['count'], reverse=True):
        print(f"  {stat['char']:<4} | {stat['count']:^7} | {stat['p']:.6f}  | {stat['info']:.4f}")


analyze_unicode_file("utf8.txt")

--- Результаты анализа Unicode файла ---
Имя файла: utf8.txt
Длина файла (n): 461275 символов
Размер алфавита |A1|: 137 символов
Точный средний размер символа: 1.42 байт/символ

--- Оценка в битах ---
Оценка суммарного количества информации I_BP(Q) = 2395316.01 бит
Дробная часть {I_BP(Q)} = 1.12e-02

--- Оценка длины архива ---
Оценка информации I_BP(Q) = 299414.50 октетов
Оценка снизу длины сжатого текста E = 299415 октетов
Оценка длины архива G_arch = 300706.00 октетов

--- Таблица характеристик символов (сортировка по символу) ---
Символ | Частота | Вероятность | Информация (бит)
------------------------------------------------------------
  
    |  13168  | 0.028547  | 5.1305
       |  70419  | 0.152662  | 2.7116
  !    |   306   | 0.000663  | 10.5579
  "    |  1079   | 0.002339  | 8.7398
  #    |   119   | 0.000258  | 11.9204
  %    |   162   | 0.000351  | 11.4754
  &    |   238   | 0.000516  | 10.9204
  '    |   568   | 0.001231  | 9.6655
  (    |  1761   | 0.003818  | 8.0331
  )

## **Задание 3**
Используем нашу функцию на следующих кодировках:

"**iso**", "**dos**", "**koi8r**", "**maccyrillic**", "**utf8**", "**utf16**", "**utf32**", "**windows**"

**Топ-4 самых частых октета (среди всех файлов):**

- **00**: Cуммарная частота 1,238,762. Этот октет доминирует из-за его использования в кодировках UTF-16 и особенно в UTF-32, где он используется для заполнения неиспользуемых байтов в представлении символов.

- **04**: Суммарная частота 606,340. Этот байт является старшим байтом для большинства символов кириллицы в кодировках UTF-16 и UTF-32.

- **20 (пробел)**: Суммарная частота 563,486. Является самым частым печатным символом во всех проанализированных текстах.

- **d0**: Суммарная частота 233,073. В основном, высокая частота обусловлена файлом 
utf8.txt, где этот байт является первым байтом двухбайтовой последовательности для большинства заглавных и строчных букв русского алфавита.

**Топ-4 самых частых октета (не ASCII):**

- **00**: Суммарная частота 1,238,762.

- **04**: Суммарная частота 606,340.

- **d0**: Суммарная частота 233,073.

- **d1**: Суммарная частота 98,425. Этот октет, как и d0, является ключевым для кодировки UTF-8, служа первым байтом для оставшейся части кириллических символов (например, от 'р' до 'я')

**Топ-4 самых частых октета (только однобайтовые кодировки)**

- **20** (пробел): Суммарная частота 352,095. Как и ожидалось, пробел является самым распространенным символом, занимая первое место во всех пяти файлах.

- **ee**: Суммарная частота 59,014. Этот октет соответствует русской букве "о" в кодировках MacCyrillic и Windows-1251, которые используются в двух из пяти анализируемых файлов.

- **e5**: Суммарная частота 54,976. Этот код представляет букву "е" в тех же кодировках, MacCyrillic и Windows-1251.

- **e8**: Суммарная частота 48,614. Этот октет соответствует букве "и" в кодировках MacCyrillic и Windows-1251.
-------------------------
**Анализ файла по варианту(7):**
Самыми частыми байтами в нашем файле являются ee, e0, e5, e8, ed и eb

В кодировке **MacCyrillic** эти коды точно соответствуют наиболее частотным буквам русского алфавита:
- **ee** -> о
- **e0** -> а
- **e5** -> е
- **e8** -> и
- **ed** -> н
- **eb** -> л
- **f1** -> с
- **f2** -> т

## Задание 4

In [15]:
def analyze_markov_file(file_path):
    try:
        # Открываем файл в бинарном режиме
        with open(file_path, 'rb') as f:
            data = f.read()
    except FileNotFoundError:
        print(f"Ошибка: Файл '{file_path}' не найден.")
        sys.exit(1)

    n = len(data)
    if n < 2:
        print("Ошибка: Файл слишком короткий для анализа пар байтов.")
        sys.exit(1)
    
    # Инициализация двумерного массива для подсчета вхождений подстрок (count(aj ak))
    pair_counts = [[0] * 256 for _ in range(256)]
    # Инициализация массива для подсчета вхождений подстрок, начинающихся с aj (count(aj*))
    start_counts = [0] * 256

    # Сбор данных
    for i in range(n - 1):
        byte1 = data[i]
        byte2 = data[i+1]
        pair_counts[byte1][byte2] += 1
        start_counts[byte1] += 1

    total_info_bits = 0.0

    # I(c1) = -log2(1/256) = -(-8) = 8 бит
    total_info_bits += 8.0

    # Информация для остальных символов на основе условной вероятности
    for i in range(n - 1):
        prev_byte = data[i]
        current_byte = data[i+1]
        
        # Условная вероятность p(ak|aj)
        if start_counts[prev_byte] > 0:
            conditional_prob = pair_counts[prev_byte][current_byte] / start_counts[prev_byte]
            if conditional_prob > 0:
                info_per_symbol = -math.log2(conditional_prob)
                total_info_bits += info_per_symbol

    print("--- Результаты анализа (Модель Маркова) ---")
    print(f"Имя файла: {file_path}")
    print(f"Длина файла (n): {n} байт")

    # Сравнение в битах
    print("\n--- Сравнение в битах ---")
    L_bits = n * 8
    print(f"Длина файла L(Q) = n * 8 = {L_bits} бит")
    print(f"Оценка суммарного количества информации I_CM1(Q) = {total_info_bits:.2f} бит")
    fractional_part = total_info_bits - int(total_info_bits)
    print(f"Дробная часть I_CM1(Q) = {fractional_part:.2e}")

    # Сравнение в октетах и оценка сжатия
    print("\n--- Сравнение в октетах и оценка сжатия ---")
    I_octets = total_info_bits / 8
    E = math.ceil(I_octets)
    
    # Оценка размера таблицы для декодирования (256x256 частот)
    table_size = 256 * 256
    G_64 = E + table_size * 8  # 64-битные частоты
    G_8 = E + table_size * 1   # 8-битные частоты

    print(f"Длина файла L(Q) = n = {n} октетов")
    print(f"Оценка информации I_CM1(Q) = {I_octets:.2f} октетов")
    print(f"Оценка снизу длины сжатого текста E = {int(E)} октетов")
    print(f"Оценка длины архива G_64 (с 64-битными частотами) = E + 256^2 * 8 = {int(G_64)} октетов")
    print(f"Оценка длины архива G_8 (с 8-битными частотами) = E + 256^2 * 1 = {int(G_8)} октетов")

analyze_markov_file("utf8.txt")

--- Результаты анализа (Модель Маркова) ---
Имя файла: utf8.txt
Длина файла (n): 764445 байт

--- Сравнение в битах ---
Длина файла L(Q) = n * 8 = 6115560 бит
Оценка суммарного количества информации I_CM1(Q) = 2020559.22 бит
Дробная часть I_CM1(Q) = 2.18e-01

--- Сравнение в октетах и оценка сжатия ---
Длина файла L(Q) = n = 764445 октетов
Оценка информации I_CM1(Q) = 252569.90 октетов
Оценка снизу длины сжатого текста E = 252570 октетов
Оценка длины архива G_64 (с 64-битными частотами) = E + 256^2 * 8 = 776858 октетов
Оценка длины архива G_8 (с 8-битными частотами) = E + 256^2 * 1 = 318106 октетов


## Задание 5

In [16]:
def analyze_markov_unicode_file(file_path):
    try:
        # Открываем файл в текстовом режиме с кодировкой UTF-8 для чтения символов
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            text = f.read()
    except FileNotFoundError:
        print(f"Ошибка: Файл '{file_path}' не найден.")
        sys.exit(1)

    n = len(text)
    if n < 2:
        print("Ошибка: Файл слишком короткий для анализа пар символов.")
        sys.exit(1)

    # Словари для подсчета частот пар и начальных символов
    pair_counts = {}
    start_counts = {}

    # Сбор данных
    for i in range(n - 1):
        prev_char = text[i]
        current_char = text[i+1]
        pair = (prev_char, current_char)

        # Подсчет пар
        pair_counts[pair] = pair_counts.get(pair, 0) + 1
        
        # Подсчет начальных символов
        start_counts[prev_char] = start_counts.get(prev_char, 0) + 1

    # Шаг 2: Расчет суммарной информации
    total_info_bits = 0.0

    # Оценка информации для первого символа:
    unique_chars = set(text)
    alphabet_size = len(unique_chars)
    if alphabet_size > 0:
        p_first_char = 1.0 / alphabet_size
        info_first_char = -math.log2(p_first_char)
        total_info_bits += info_first_char

    # Информация для остальных символов на основе условной вероятности
    for i in range(n - 1):
        prev_char = text[i]
        current_char = text[i+1]
        pair = (prev_char, current_char)

        if start_counts[prev_char] > 0:
            conditional_prob = pair_counts[pair] / start_counts[prev_char]
            if conditional_prob > 0:
                info_per_symbol = -math.log2(conditional_prob)
                total_info_bits += info_per_symbol

    print("--- Результаты анализа Unicode (Модель Маркова) ---")
    print(f"Имя файла: {file_path}")
    print(f"Длина файла (n): {n} символов")
    print(f"Размер алфавита |A1|: {alphabet_size} символов")
    
    # Сравнение в битах
    print("\n--- Оценка в битах ---")
    print(f"Оценка суммарного количества информации I_CM1(Q) = {total_info_bits:.2f} бит")
    fractional_part = total_info_bits - int(total_info_bits)
    print(f"Дробная часть I_CM1(Q) = {fractional_part:.2e}")

    # Оценка длины архива
    print("\n--- Оценка длины архива ---")
    I_octets = total_info_bits / 8
    E = math.ceil(I_octets)

    # Оценка размера таблицы для декодирования (пары символ-частота)
    # Используем тот же подход, что и в Л2.2
    # Точный расчет среднего размера символа
    total_symbol_bytes = sum(len(char.encode('utf-8')) for char in unique_chars)
    avg_symbol_size = total_symbol_bytes / alphabet_size if alphabet_size > 0 else 0

    # Размер таблицы = (количество уникальных пар * (средний размер 2-х символов + размер частоты))
    table_size_estimate = len(pair_counts) * (2 * avg_symbol_size + 8)
    G_arch = E + table_size_estimate

    print(f"Оценка информации I_CM1(Q) = {I_octets:.2f} октетов")
    print(f"Оценка снизу длины сжатого текста E = {int(E)} октетов")
    print(f"Оценка длины архива G_arch = {G_arch:.2f} октетов")

analyze_markov_unicode_file("utf8.txt")

--- Результаты анализа Unicode (Модель Маркова) ---
Имя файла: utf8.txt
Длина файла (n): 461275 символов
Размер алфавита |A1|: 137 символов

--- Оценка в битах ---
Оценка суммарного количества информации I_CM1(Q) = 1736402.79 бит
Дробная часть I_CM1(Q) = 7.93e-01

--- Оценка длины архива ---
Оценка информации I_CM1(Q) = 217050.35 октетов
Оценка снизу длины сжатого текста E = 217051 октетов
Оценка длины архива G_arch = 251511.01 октетов
